A good tool makes simple things easy and complex things possible.

If you don't know which solution is better, start with the one that's simpler.

In [ ]:
import os
try:
    from yaml import load, CLoader as Loader
except:
    from yaml import load, Loader

data = load(open('env/local-env.yml'), Loader=Loader)
os.environ.update({key: str(val) for key, val in data.items()})

from hemlock import push_app_context

app = push_app_context()

In [23]:
from hemlock import Check, Debug as D, Page, push_app_context
from hemlock.tools import chromedriver

app = push_app_context()

driver = chromedriver()

p = Page(
    Check(
        '<p>Click "Yes".</p>', 
        ['Yes', 'No'], 
        multiple=True,
        debug=D.clear_choices()
    )
)
p.debug.pop()
p.preview(driver)._debug(driver)

<Page 10>

In [24]:
import os
driver.close()
[os.remove(tmpfile) for tmpfile in app.tmpfiles]

[None]